In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (20, 10)

## Introduction 

In [2]:
smartphones = pd.read_csv('smartphone_raw.csv')

In [3]:
smartphones.shape

(338018, 5)

In [4]:
smartphones.head()

,event_time,event_type,category_id,price,user_id
0,2019-10-01 00:02:14 UTC,purchase,2053013555631882655,130.76,543272936
1,2019-10-01 00:04:37 UTC,purchase,2053013555631882655,642.69,551377651
2,2019-10-01 00:10:08 UTC,purchase,2053013555631882655,515.67,524325294
3,2019-10-01 00:14:14 UTC,purchase,2053013555631882655,463.31,555083442
4,2019-10-01 02:19:10 UTC,purchase,2053013555631882655,736.18,515246296


In [5]:
smartphones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338018 entries, 0 to 338017
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   event_time   338018 non-null  object 
 1   event_type   338018 non-null  object 
 2   category_id  338018 non-null  int64  
 3   price        338018 non-null  float64
 4   user_id      338018 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 12.9+ MB


## Data cleaning

In [6]:
round((smartphones.isnull().sum() / smartphones.shape[0]) * 100, 2)

event_time     0.0
event_type     0.0
category_id    0.0
price          0.0
user_id        0.0
dtype: float64

Данные чистые. Заполнять искусственно не нужно.

## Data Preparation

Удалим время у фичи `event_time`, и переименуем ее в `event_date`

In [7]:
smartphones['event_time'] = pd.to_datetime(smartphones['event_time'])

In [8]:
smartphones['event_time'] = smartphones['event_time'].map(lambda x: 10000*x.year + 100*x.month + x.day)

In [9]:
smartphones = smartphones.rename(columns={'event_time': 'event_date'})

In [10]:
smartphones.head()

,event_date,event_type,category_id,price,user_id
0,20191001,purchase,2053013555631882655,130.76,543272936
1,20191001,purchase,2053013555631882655,642.69,551377651
2,20191001,purchase,2053013555631882655,515.67,524325294
3,20191001,purchase,2053013555631882655,463.31,555083442
4,20191001,purchase,2053013555631882655,736.18,515246296


Удалю колонки, которые нам уже ненужные колонки - `category_id`

In [11]:
smartphones = smartphones.drop('category_id', axis=1)

In [12]:
smartphones.head()

,event_date,event_type,price,user_id
0,20191001,purchase,130.76,543272936
1,20191001,purchase,642.69,551377651
2,20191001,purchase,515.67,524325294
3,20191001,purchase,463.31,555083442
4,20191001,purchase,736.18,515246296


In [13]:
smartphones.to_csv('smartphones_cleaned.csv', index=False)

И снова сократил с 23,7 MB до 11 MB. И притом оставили только нужные нам данные.

### RFM

Для кластеризации пользователей я буду использовать сегментацию **RFM**.

**RFM** (**R**ecency **F**requency **M**onetary) - разделение клиентов на сегменты от степени их лоялности.

У такой сегментации всего 3 колонки:

**R**ecency (давность) - давность прошлой сделки, сколько времени прошло с прошлой покупки. Предполагается, чем меньше эта метрика, тем больше вероятность будущей, повторной покупки.

**F**requency (частота) - кол-во покупок. Больше покупок, больше вероятность возвращение клиента.

**M**onetary (денежная масса, деньги) - сумма сделок. Чем больше потратил клиент, тем больше вероятность возвращение клиента.

### Data Transformation 

In [14]:
# Для начала найдем уникальных юзеров 

users = pd.DataFrame({'user_id': smartphones['user_id'].unique()})

In [15]:
users.head()

,user_id
0,543272936
1,551377651
2,524325294
3,555083442
4,515246296


In [16]:
# Их кстати 
users.shape[0]

160437

### Соберем дату последней покупке юзеров

In [17]:
%%time
for user in users['user_id']:
    users.loc[users['user_id'] == user, 'last_ordered_date'] = \
    smartphones[smartphones['user_id'] == user]['event_date'].iloc[-1]

CPU times: user 3min 50s, sys: 1.18 s, total: 3min 52s
Wall time: 3min 53s


In [18]:
users.head()

,user_id,last_ordered_date
0,543272936,20191031.0
1,551377651,20191025.0
2,524325294,20191008.0
3,555083442,20191005.0
4,515246296,20191024.0


In [19]:
users['user_id'] = users['user_id'].astype(int)
users['last_ordered_date'] = users['last_ordered_date'].astype(int)

In [20]:
users.head()

,user_id,last_ordered_date
0,543272936,20191031
1,551377651,20191025
2,524325294,20191008
3,555083442,20191005
4,515246296,20191024


### Теперь соберем информацию о кол-во покупок.

In [ ]:
%%time
for user in users['user_id']:
    users.loc[users['user_id'] == user, 'n_orders'] = \
    smartphones[smartphones['user_id'] == user]['event_type'].count()

In [ ]:
users.head()

In [ ]:
users['n_orders'] = users['n_orders'].astype(int)

### И последнее потраченная сумма.

In [ ]:
%%time
for user in users['user_id']:
    users.loc[users['user_id'] == user, 'amount'] = \
    smartphones[smartphones['user_id'] == user]['price'].sum()

In [ ]:
users.head()

In [ ]:
users['amount'] = users['amount'].astype(int)

In [ ]:
users.head()

И так последний шрихт. `last_ordered_date` преобразовать в `recency`,  то есть вычесть самый-самый последний день, указанные в данных, из последнего совершения покупки определенного клиента.

In [ ]:
max_date = smartphones['event_date'].max()
max_date

In [ ]:
users['order_time_offset'] = (users['last_ordered_date'] - max_date) * -1

In [ ]:
users.head()

In [ ]:
users = users.drop('last_ordered_date', axis=1)

In [ ]:
users.head()

In [ ]:
rfm = users.drop('user_id', axis=1)

In [ ]:
rfm.head()

### Переименование и упорядочевания колонок.
Для красоты и понятности переименую колонки и упорядочую как в аббревиатуре. 

In [ ]:
rfm = rfm.rename( \
    columns={'n_orders': 'Recency', 'order_time_offset':'Frequency', 'amount': 'Monetary'}
)

In [ ]:
rfm.head()

In [ ]:
rfm = rfm[['Recency', 'Frequency', 'Monetary']]

In [ ]:
rfm.head()

Визуализация величин. Распределение по гисто и QQPlot.

In [ ]:
fig, axes = plt.subplots(1, 3)
rec_hist = sns.distplot(ax=axes[0], a=rfm['Recency'], kde=True)
rec_hist.set_title('Distribution of `Recency`')

fre_hist = sns.distplot(ax=axes[1], a=rfm['Frequency'], kde=True)
fre_hist.set_title('Distribution of `Frequency`')

mon_hist = sns.distplot(ax=axes[2], a=rfm['Monetary'], kde=True)
mon_hist.set_title('Distribution of `Monetary`');

In [ ]:
from scipy import stats

fig, axes = plt.subplots(1, 3)
stats.probplot(x=rfm['Recency'], dist="norm", plot=axes[0])
stats.probplot(x=rfm['Frequency'], dist="norm", plot=axes[1])
stats.probplot(x=rfm['Monetary'], dist="norm", plot=axes[2]);

In [ ]:
fig, axes = plt.subplots(1, 3)
rec_box = sns.boxplot(ax=axes[0], y=rfm['Recency'])
rec_box.set_title('Boxplot of `Recency`')

fre_box = sns.boxplot(ax=axes[1], y=rfm['Frequency'])
fre_box.set_title('Boxplot of `Frequency`')

mon_box = sns.boxplot(ax=axes[2], y=rfm['Monetary'])
mon_box.set_title('Boxplot of `Monetary`');

### Скалирование величин.
Многие алгоритмы кластеризации под капотом вычисляют дистанции (Euclidean, Manhattan). Поэтому скалирование величин **обязательный** гость программы.

Из-за того, что в `Recency` и `Monetary` большой разброс значений, я буду использовать `MinMax` скалирование.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

for column in rfm.columns:
    rfm[column] = scaler.fit_transform(rfm[[column]])

In [ ]:
rfm.head()

In [ ]:
del scaler

In [ ]:
rfm_scaled = rfm.copy()

### Удаление выбросов.

In [ ]:
for column in rfm.columns:
    rfm_scaled = rfm_scaled[(rfm_scaled[column] < 3) & (rfm_scaled[column] > -3)]

In [ ]:
rfm_scaled.head()

In [ ]:
fig, axes = plt.subplots(1, 3)
rec_hist = sns.distplot(ax=axes[0], a=rfm_scaled['Recency'], kde=True)
rec_hist.set_title('Distribution of Scaled `Recency`')

fre_hist = sns.distplot(ax=axes[1], a=rfm_scaled['Frequency'], kde=True)
fre_hist.set_title('Distribution of Scaled `Frequency`')

mon_hist = sns.distplot(ax=axes[2], a=rfm_scaled['Monetary'], kde=True)
mon_hist.set_title('Distribution of Scaled `Monetary`');

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(rfm_scaled['Recency'], rfm_scaled['Monetary'], rfm_scaled['Frequency'])
ax.set_xlabel("Recency")
ax.set_ylabel("Monetary")
ax.set_zlabel("Frequency")
plt.show()

### Выбор и тренировка моделей.

Выбор моделей, я конечно же начну с самого простого и популярного алгортима K-means.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Найдем кол-во кластеров с помощью `Elbow curve`

results = []
range_clust = range(2, 15)

for num in range_clust:
    kmeans = KMeans(n_clusters=num)
    kmeans.fit(rfm_scaled)
    
    results.append({'N_clusters': num, 'Inertia': kmeans.inertia_})

In [ ]:
results = pd.DataFrame(results)

In [ ]:
sns.lineplot(data=results, x='N_clusters', y='Inertia');

***По методу Elbow выбираем ответ 4.*** <img src='https://miro.medium.com/max/1400/1*eVyOdx4gIcGWQ3lF4xAu6g.png' width='400' heigh='200'/>

In [ ]:
kmeans_elbow = KMeans(n_clusters=4)
kmeans_elbow.fit(rfm_scaled)

In [ ]:
rfm_scaled['Cluster_id'] = kmeans_elbow.labels_

In [ ]:
rfm_scaled['Cluster_id'] = rfm_scaled['Cluster_id'] + 1

In [ ]:
rfm_scaled.head()

### 

In [ ]:
fig, axes = plt.subplots(1,3)

rec_cluster = sns.boxplot(ax=axes[0], x='Cluster_id', y='Recency', data=rfm_scaled);
rec_cluster.set_title("Clustered `Recency` boxplot")
freq_cluster = sns.boxplot(ax=axes[1], x='Cluster_id', y='Frequency', data=rfm_scaled);
freq_cluster.set_title("Clustered `Frequency` boxplot")
mon_cluster = sns.boxplot(ax=axes[2], x='Cluster_id', y='Monetary', data=rfm_scaled);
mon_cluster.set_title("Clustered `Monetary` boxplot");

### Выводы

Недавно покупали товар пользователи из кластера 1, 2, 3 

Чаще всего покупают 3 кластер, 1 кластер и некоторые пользователи из 4 кластера

Юзеры из 4 кластер айди покупают больше всех, в остальных кластерах покупатели тратять примерно одиниково, не считая выбросов.

Может 4 кластер - это оптовики? Покупают много часто и последнюю покупку совершали давно. Может им предложить отдельную цену за товар. Тогда мы сможем их вернуть и продать им еще больше смартфонов.